Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload 20newsgroups  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

category_list = []
for i in range(46):
    category_list.append(0)
print(len(category_list))

for i in train_labels:
    category_list[i] += 1

extracted_index = []

for i, v in enumerate(category_list):
    if v >= 100:
#         print(i, v)
        extracted_index.append(i)


extracted_train_data = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_data.append(train_data[i])

extracted_test_data = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_data.append(test_data[i])

extracted_train_labels = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_labels.append(train_labels[i])

extracted_test_labels = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_labels.append(test_labels[i])

word_to_index = reuters.get_word_index()

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

#문장 추출
to_txt=[]
for i in range(7828):
  
    to_txt.append(' '.join([index_to_word[index] for index in extracted_train_data[i]])[6:])
    
for i in range(1934):
    to_txt.append(' '.join([index_to_word[index] for index in extracted_test_data[i]])[6:])

46


In [3]:
y=extracted_train_labels+extracted_test_labels

encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:7828]
y_test=label[7828:]

In [4]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## 2) textcuboid 분류

In [5]:
textcuboid=np.load('./1-Channel textcuboid_reuters(bert).npy')

In [6]:
textcuboid_test=np.load('./1-Channel textcuboid_test_reuters(bert).npy')

In [91]:
import keras
from tensorflow.keras import layers

In [92]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [93]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(119, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(1024, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(12, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [94]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [95]:
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 119, 768)]        0         
                                                                 
 conv1d_11 (Conv1D)          (None, 119, 1024)         787456    
                                                                 
 global_max_pooling1d_11 (Gl  (None, 1024)             0         
 obalMaxPooling1D)                                               
                                                                 
 dense_22 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_11 (Dropout)        (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 12)                12300     
                                                          

In [96]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

In [97]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
27/27 [==============================] - 2s 46ms/step - loss: 6.9755 - accuracy: 0.4940 - val_loss: 1.0871 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 2/20
27/27 [==============================] - 1s 34ms/step - loss: 0.9845 - accuracy: 0.7066 - val_loss: 0.8032 - val_accuracy: 0.7890 - lr: 0.0010
Epoch 3/20
27/27 [==============================] - 1s 34ms/step - loss: 0.7450 - accuracy: 0.7963 - val_loss: 0.6588 - val_accuracy: 0.8170 - lr: 0.0010
Epoch 4/20
27/27 [==============================] - 1s 34ms/step - loss: 0.5777 - accuracy: 0.8462 - val_loss: 0.5777 - val_accuracy: 0.8530 - lr: 0.0010
Epoch 5/20
27/27 [==============================] - 1s 34ms/step - loss: 0.4725 - accuracy: 0.8806 - val_loss: 0.5444 - val_accuracy: 0.8590 - lr: 0.0010
Epoch 6/20
27/27 [==============================] - 1s 34ms/step - loss: 0.3813 - accuracy: 0.9033 - val_loss: 0.5093 - val_accuracy: 0.8720 - lr: 0.0010
Epoch 7/20
27/27 [==============================] - 1s 34ms/step - loss: 0.3

In [98]:
model.evaluate(textcuboid_test,one_hot_test_labels)

61/61 [==============================] - 0s 4ms/step - loss: 0.5679 - accuracy: 0.8775


[0.5678754448890686, 0.8774560689926147]

In [99]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

61/61 [==============================] - 0s 4ms/step - loss: 0.5275 - accuracy: 0.8764


[0.5275211334228516, 0.8764219284057617]

In [100]:
np.average([87.2, 87.3, 87.8])

87.43333333333334